In [1]:
import pandas as pd
import os
import pickle
import importlib
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error



In [3]:

package_name = "mlflow"
try:
    importlib.import_module(package_name)
    print(f"{package_name} is already installed.")
except ModuleNotFoundError:
    print(f"{package_name} not found. Installing...")
    %pip install {package_name}

mlflow is already installed.
